<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/leerazo/neo4j_aura_api/blob/main/02-using_aura_api/02-using_aura_api.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/leerazo/neo4j_aura_api/blob/main/02-using_aura_api/02-using_aura_api.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/leerazo/neo4j_aura_api/blob/main/02-using_aura_api/02-using_aura_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">Open in Vertex AI Workbench
    </a>
</td>
</table>

# Authenticating and Exploring the Neo4j Aura API Environment

Let's begin by installing and importing all the needed libraries.

This cell will automatically restart the kernel after installation.

If you see a message that "Your session crashed for an unknown reason" during this step, you can safely ignore that.

In [ ]:
%pip install python-dotenv

# Restart the kernel after installing libraries
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
from dotenv import dotenv_values
from urllib.parse import urljoin
import subprocess
import datetime
import json

## Step 1: Authenticate API and get a bearer token for access

Upload the API credential file you created earlier. If you are using Colab you can click the button below to locate and upload the file. If you are using another platform you can do this manually.

In [ ]:
from google.colab import files
uploaded = files.upload()

Now we'll extract the credentials from the file into a dictionary for further use.

In [ ]:
filename = list(uploaded.keys())[0]
print(filename)

aura_creds = dotenv_values(filename)
for item in aura_creds:
  print(item, '=', aura_creds[item])

Let's define a helper function to generate a bearer token for access using the downloaded credentials.

In [ ]:
def refresh_token(api_creds, api_base):
    api_endpoint = urljoin(api_base, '/oauth/token')

    curl_cmd = "curl --request POST '{}' --user '{}:{}' --header 'Content-Type: application/x-www-form-urlencoded' --data-urlencode 'grant_type=client_credentials'".format(api_endpoint, api_creds['CLIENT_ID'], api_creds['CLIENT_SECRET'], api_creds['CLIENT_NAME'])
    result = json.loads(subprocess.check_output(curl_cmd, shell=True))
    access_token = result['access_token']
    expires_in = result['expires_in']

    now = datetime.datetime.now()
    expiration = (now + datetime.timedelta(0, expires_in)).isoformat()

    bearer_token = {
        'access_token': access_token,
        'expiration': expiration
    }

    # Save the bearer token to a file
    token_file = 'api_bearer_token'
    with open(token_file, "w") as outfile:
        json.dump(bearer_token, outfile, indent=4)

    access_token = bearer_token['access_token']
    return access_token

api_base = 'https://api.neo4j.io/'

access_token = refresh_token(aura_creds, api_base)
print('access_token:', access_token)

## Step 2: Explore the Aura tenants

Now that we're authenticated, let's take a look around and see what we have access to.

Let's define a few more helper functions.

In [ ]:
def list_tenants(access_token, api_base, tenant_id=None):
    #api_endpoint = 'https://api.neo4j.io/v1/tenants'

    api_endpoint = urljoin(api_base, '/v1/tenants')

    aura_tenants = {}

    if tenant_id:
        api_endpoint += '/' + tenant_id

    print()
    print('api_endpoint:', api_endpoint)
    print()

    list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    list_tenants = dict(json.loads(subprocess.check_output(list_cmd, shell=True)))['data']
    for item in list_tenants:
        aura_tenants[item['id']] = {}
        aura_tenants[item['id']]['id'] = item['id']
        aura_tenants[item['id']]['name'] = item['name']

    return aura_tenants


def tenant_info(access_token, api_base, tenant_id):
    api_endpoint = urljoin(api_base, '/v1/tenants/' + tenant_id)
    tenant_data = {}
    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)
    tenant_info = json.loads(subprocess.check_output(info_cmd, shell=True))['data']

    for item in tenant_info:
        tenant_data[tenant_info['id']] = {}
        tenant_data[tenant_info['id']]['tenant_id'] = tenant_info['id']
        tenant_data[tenant_info['id']]['tenant_name'] = tenant_info['name']
        tenant_data[tenant_info['id']]['instance_configurations'] = list(tenant_info['instance_configurations'])

    return tenant_data


Now let's take a look at the environment, beginning with a list of tenants we have access to.

In [ ]:
aura_tenants = list_tenants(access_token, api_base)
print()
print('Aura Tenants:')
print('-------------')
for tenant in aura_tenants:
    print(aura_tenants[tenant])
print()

We can also get a take a more detailed look into one of the tenants.

In [ ]:
sample_tenant_id = next(iter(aura_tenants))
print('sample_tenant_id:', sample_tenant_id)
print()

tenant_details = tenant_info(access_token, api_base, sample_tenant_id)

print('Tenant Details:')
print('---------------')
for item in tenant_details[sample_tenant_id]:
    print(item, '=', tenant_details[sample_tenant_id][item])

The following cell will list out all of the supported Aura configurations available in our sample tenant.

In [ ]:
for config in tenant_details[sample_tenant_id]['instance_configurations']:
    print(config)

## Step 3: Examine existing Aura instances

Let's take a look and see if there are already any Aura instances running in oru tenant. We'll need to define a couple more helper functions first.


In [ ]:
def list_instances(access_token, api_base, tenant_id=None):
    api_endpoint = urljoin(api_base, '/v1/instances')
    aura_instances = {}
    list_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    list_instances = json.loads(subprocess.check_output(list_cmd, shell=True))['data']
    for instance in list_instances:
        aura_instances[instance['id']] = {}
        aura_instances[instance['id']]['instance_id'] = instance['id']
        aura_instances[instance['id']]['instance_name'] = instance['name']
        aura_instances[instance['id']]['cloud_provider'] = instance['cloud_provider']
        aura_instances[instance['id']]['tenant_id'] = instance['tenant_id']

    return aura_instances

def instance_info(access_token, api_base, instance_id):
    api_endpoint = urljoin(api_base, '/v1/instances/' + instance_id)
    instance_data = {}

    info_cmd = "curl -s -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'".format(api_endpoint, access_token)

    api_response = json.loads(subprocess.check_output(info_cmd, shell=True))

    if 'data' in api_response:
        response_data = api_response['data']
        instance_status = response_data['status']

    return response_data

Now let's get a list of all of the instances running.


In [ ]:
aura_instances = list_instances(access_token, api_base)
print('aura_instances:')
for instance in aura_instances:
    print(instance, '=', aura_instances[instance])

Let's take a closer look at one of them.

In [ ]:
instance_details = instance_info(access_token, api_base, next(iter(aura_instances)))
print('instance_details:')
print()
for item in instance_details:
    print('\t' + item, '=', instance_details[item])

## Congratulations!
You get the idea, now you're ready to try and [deploy a new Aura instance](../03-deploying_aura).